In [18]:
pip install pandas numpy scikit-learn tensorflow matplotlib seaborn



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import pandas as pd

# Load the dataset
job_postings_df = pd.read_csv(r"C:\Users\91844\Desktop\swipe-hire\linkedin_job_postings.csv")

# Display the first few rows
print(job_postings_df.head())


                                            job_link  \
0  https://www.linkedin.com/jobs/view/account-exe...   
1  https://www.linkedin.com/jobs/view/registered-...   
2  https://www.linkedin.com/jobs/view/restaurant-...   
3  https://www.linkedin.com/jobs/view/independent...   
4  https://www.linkedin.com/jobs/view/group-unit-...   

             last_processed_time got_summary got_ner is_being_worked  \
0   2024-01-21 07:12:29.00256+00           t       t               f   
1   2024-01-21 07:39:58.88137+00           t       t               f   
2  2024-01-21 07:40:00.251126+00           t       t               f   
3  2024-01-21 07:40:00.308133+00           t       t               f   
4  2024-01-19 09:45:09.215838+00           f       f               f   

                                           job_title  \
0  Account Executive - Dispensing (NorCal/Norther...   
1                 Registered Nurse - RN Care Manager   
2               RESTAURANT SUPERVISOR - THE FORKLIFT   
3     

In [20]:
job_skills_df = pd.read_csv(r"C:\Users\91844\Desktop\swipe-hire\job_skills.csv")

In [21]:
# Merge the datasets based on the 'job_link' column
merged_df = pd.merge(job_postings_df, job_skills_df, on='job_link', how='inner')

# Display the merged dataset to verify
print(merged_df.head())


                                            job_link  \
0  https://www.linkedin.com/jobs/view/account-exe...   
1  https://www.linkedin.com/jobs/view/registered-...   
2  https://www.linkedin.com/jobs/view/restaurant-...   
3  https://www.linkedin.com/jobs/view/independent...   
4  https://www.linkedin.com/jobs/view/registered-...   

             last_processed_time got_summary got_ner is_being_worked  \
0   2024-01-21 07:12:29.00256+00           t       t               f   
1   2024-01-21 07:39:58.88137+00           t       t               f   
2  2024-01-21 07:40:00.251126+00           t       t               f   
3  2024-01-21 07:40:00.308133+00           t       t               f   
4  2024-01-21 08:08:19.663033+00           t       t               f   

                                           job_title  \
0  Account Executive - Dispensing (NorCal/Norther...   
1                 Registered Nurse - RN Care Manager   
2               RESTAURANT SUPERVISOR - THE FORKLIFT   
3     

In [22]:
# Clean column names: strip whitespace and convert to lowercase
merged_df.columns = merged_df.columns.str.strip().str.lower()

# Now print the cleaned column names to confirm
print(merged_df.columns)


Index(['job_link', 'last_processed_time', 'got_summary', 'got_ner',
       'is_being_worked', 'job_title', 'company', 'job_location', 'first_seen',
       'search_city', 'search_country', 'search_position', 'job_level',
       'job_type', 'job_skills'],
      dtype='object')


In [23]:
# Fill missing values with a placeholder
merged_df['job_title'].fillna('Unknown', inplace=True)
merged_df['company'].fillna('Unknown', inplace=True)
merged_df['job_location'].fillna('Unknown', inplace=True)
merged_df['job_skills'].fillna('No Skills Listed', inplace=True)
merged_df['job_type'].fillna('No type available', inplace=True)

# Check for remaining missing values
print(merged_df.isnull().sum())


C:\Users\91844\AppData\Local\Temp\ipykernel_40444\486724544.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['job_title'].fillna('Unknown', inplace=True)
C:\Users\91844\AppData\Local\Temp\ipykernel_40444\486724544.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

job_link               0
last_processed_time    0
got_summary            0
got_ner                0
is_being_worked        0
job_title              0
company                0
job_location           0
first_seen             0
search_city            0
search_country         0
search_position        0
job_level              0
job_type               0
job_skills             0
dtype: int64


In [24]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import nltk
print(nltk.__version__)


3.9.1


In [26]:
pip install re

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement re (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for re


In [27]:
import nltk
print(nltk.data.path)


['C:\\Users\\91844/nltk_data', 'c:\\Users\\91844\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data', 'c:\\Users\\91844\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data', 'c:\\Users\\91844\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data', 'C:\\Users\\91844\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [28]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Clean and preprocess the job descriptions and skills
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove non-alphanumeric characters (except spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

merged_df['job_title'] = merged_df['job_title'].apply(preprocess_text)
merged_df['job_skills'] = merged_df['job_skills'].apply(preprocess_text)

# Check the cleaned data
print(merged_df[['job_title', 'job_link', 'job_skills']].head())


                                           job_title  \
0  account executive  dispensing norcalnorthern n...   
1                  registered nurse  rn care manager   
2                restaurant supervisor  the forklift   
3                      independent real estate agent   
4                                registered nurse rn   

                                            job_link  \
0  https://www.linkedin.com/jobs/view/account-exe...   
1  https://www.linkedin.com/jobs/view/registered-...   
2  https://www.linkedin.com/jobs/view/restaurant-...   
3  https://www.linkedin.com/jobs/view/independent...   
4  https://www.linkedin.com/jobs/view/registered-...   

                                          job_skills  
0  medical equipment sales key competitors termin...  
1  nursing bachelor of science in nursing masters...  
2  restaurant operations management inventory man...  
3  real estate customer service sales negotiation...  
4  nursing bsn medical license virtual rn nursing..

In [29]:
# Vectorize the 'Job Description' and 'Skills' columns
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Apply TF-IDF to 'Job Description' column
job_description_tfidf = tfidf_vectorizer.fit_transform(merged_df['job_type'])

# Apply TF-IDF to 'Skills' column
skills_tfidf = tfidf_vectorizer.fit_transform(merged_df['job_skills'])

# Check the dimensions of the vectors
print(job_description_tfidf.shape, skills_tfidf.shape)


(1296381, 3) (1296381, 215289)


In [30]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def recommend_jobs_by_input(user_input, df, similarity_matrix, top_n=5):
    # Check if the user input is a job title or skill(s)
    if any(title.lower() in user_input.lower() for title in df['job_title']):
        # Input is a job title, recommend based on job descriptions
        job_index = df[df['job_title'].str.contains(user_input, case=False, na=False)].index[0]
        similarity_scores = similarity_matrix[job_index]
        similar_jobs = similarity_scores.argsort()[-top_n-1:-1][::-1]
        return df.iloc[similar_jobs][['job_title', 'company', 'job_location', 'job_link']]
    
    else:
        # Input is skills, recommend based on skills
        skills_tfidf = TfidfVectorizer(stop_words='english').fit_transform(df['job_skills'])
        skills_similarity = cosine_similarity(skills_tfidf, skills_tfidf)
        
        # Find the best match for skills
        skills_similarity_scores = skills_similarity[0]
        similar_jobs = skills_similarity_scores.argsort()[-top_n-1:-1][::-1]
        return df.iloc[similar_jobs][['job_title', 'company', 'job_location', 'job_link']]



In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse


# Limit dataset size for testing (using top 1000 jobs to avoid memory overload)
df_sample = merged_df.head(10000)  # Limit to top 1000 rows for testing

# Preprocess job descriptions (job titles) for cosine similarity calculation
job_description_tfidf = TfidfVectorizer(stop_words='english').fit_transform(df_sample['job_title'])

# Convert to sparse format (to save memory)
job_description_tfidf_sparse = scipy.sparse.csr_matrix(job_description_tfidf)

# Calculate cosine similarity on sparse matrix
job_description_similarity = cosine_similarity(job_description_tfidf_sparse, job_description_tfidf_sparse)

# Function to recommend jobs based on user input (either job title or skills)
def recommend_jobs_by_input(user_input, df, similarity_matrix, top_n=5):
    # Check if the user input is a job title or skill(s)
    if any(title.lower() in user_input.lower() for title in df['job_title']):
        # Input is a job title, recommend based on job descriptions
        job_index = df[df['job_title'].str.contains(user_input, case=False, na=False)].index[0]
        similarity_scores = similarity_matrix[job_index]
        similar_jobs = similarity_scores.argsort()[-top_n-1:-1][::-1]
        return df.iloc[similar_jobs][['job_title', 'company', 'job_location', 'job_link']]
    
    else:
        # Input is skills, recommend based on skills
        skills_tfidf = TfidfVectorizer(stop_words='english').fit_transform(df['job_skills'])
        skills_similarity = cosine_similarity(skills_tfidf, skills_tfidf)
        
        # Find the best match for skills
        skills_similarity_scores = skills_similarity[0]
        similar_jobs = skills_similarity_scores.argsort()[-top_n-1:-1][::-1]
        return df.iloc[similar_jobs][['job_title', 'company','job_skills', 'job_location', 'job_link']]

# Taking input from the user for job title or skills
user_input = input("Enter a job title or skills to find relevant job links: ")

# Get recommendations based on user input
recommended_jobs = recommend_jobs_by_input(user_input, df_sample, job_description_similarity)

# Display the recommended jobs
print("\nRecommended Jobs:")
print(recommended_jobs.to_string(index=False))  



Recommended Jobs:
                                              job_title                      company                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 